In [1]:
import netCDF4 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import datetime as dt
import xarray as xr
import pyproj
import requests

# for suitable font size in plots
font = {'family' : 'serif', 
        #'weight' : 'bold', 
        'size'   : 19} 
plt.rc('font', **font)

# READ data

In [10]:
"""USING XARRAY"""

# Start date of dataset
start_year = 2017
start_month = 1
start_day = 1
start_hour = 0

# End data of dataset
end_year = 2017
end_month = 1
end_day = 1
end_hour = 12

#analysis times
times = ['00', '06', '12','18']

#Number of days
num_days = dt.datetime(end_year,end_month,end_day, end_hour, 0,0) - \
dt.datetime(start_year,start_month,start_day, start_hour,0,0)

#number of analysis times
num_times = len(times)

#define longitude and latitude for data
# Bergen, Geofysisk Institutt
lon= 5.331824833939899
lat= 60.3837933413571
dist_threshold = 3000

#Meteorolgisk Insitutt, Blindern, Oslo
#lon = 10.720725373955348
#lat = 59.94299117739529

count=0

#loop through days
for i in range(0,num_days.days):
    YEAR = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%Y')
    MONTH  = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%m')
    DAY = dt.datetime(start_year,start_month,start_day + i, 0, 0, 0).strftime('%d')
    
    #loop through analysis times
    for j in range(num_times):
        #Open dataset
        URL = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/' + YEAR + '/' \
        + MONTH + '/' + DAY + '/' + 'meps_subset_2_5km_' + YEAR + MONTH + DAY + 'T' \
        + times[j] + 'Z.nc'
        
        
        """Check if data exists"""
        request = requests.get(URL + '.html')
        #If status is 200, the data exist
        if request.status_code != 200:
            print('WARNING: Data for', URL, ' does not exist. Fill with NaNs')
            if count == 0:
                print('WARNING: Dataset will start at next model run')
                continue
            else: 
                # create subset filled with nan.
                # create a time range from this forecast reference time until 48h into the future
                date = datetime(YEAR, MONTH, DAY, times[j], 0, 0, 000000000)
                time = pd.date_range(start=date, end= date + dt.timedelta(hours=48), periods = 67)
                # create new dataset
                newdata = xr.Dataset(data_vars=dict(met_forecast_ref_time = date,\
                                                precipitation_amount_acc=precipitation_amount_acc*np.nan,\
                                                air_pressure_at_sea_level=air_pressure_at_sea_level*np.nan,\
                                                air_temperature_2m=air_temperature_2m*np.nan),\
                                 coords = ds.coords,\
                                 attrs = ds.attrs )
                # replace the Index Variable 'time' (which is the index variable of time[j-1])
                # with the previously made time range for this particular model run
                newdata.set_index(time = time)
                #combine previous data with new data along the 'model_run' dimension
                data = xr.concat([data, newdata], dim ='model_runs')
                continue
        else:
            print('Read data from:', URL)
            ds = xr.open_dataset(URL)
        
        """Find nearest grid to X,Y"""
        # Compute projected coordinates of lat/lon point
        proj = pyproj.Proj(ds.projection_lambert.proj4) 
        X,Y = proj(lon,lat)
        
        # Find row numer of chosen x and y coordinate (for the given lat/lon)
        ix = ds.indexes['x'].get_loc(X,method='nearest')
        iy = ds.indexes['y'].get_loc(Y,method='nearest')
        
        #Get forecast reference time. Will be used in for each dimension of 'model_runs'
        met_forecast_ref_time = ds.variables['forecast_reference_time']
        
        
        """Find the surrounding grids to chosen lat/lon point and take the mean"""       
        if count == 0: #only do this the frist round
            # Selesct a subset of points around X, Y
            check_points=ds.isel(x=slice(ix-5, ix+5), y=slice(iy-5,iy+5))
            indx_x = []
            indx_j = []
            for i in range(len(check_points.x)):
                for j in range(len(check_points.y)):
                    # Calc distance between the points in x and y dir, and calc distance
                    dx = abs(check_points.x[i] - X)
                    dy = abs(check_points.y[j] - Y)
                    distance = np.sqrt(dx**2 + dy**2)
                    #if distance is within the threshoLD, add the indexes to list
                    if distance <= dist_threshold:
                        indx_x.append(ix - 5 + i)
                        indx_y.append(iy -5 + j)
                    else:
                        continue
        else:
            continue
            
        # select data from the grids within threshold distance, and take the mean over this dimension
        xsel=ds.x[indx_x]
        ysel=ds.y[indx_y]
        ds = ds.sel(x=xsel,y=ysel)        
        
        """ ####### GETTING *ALL* THE VARIABLES ####### """
        # selecting data from one specific lon, lat location, and one specific ensamble member
        # ds.air_temperature_2m.sel(x=X,y=Y,method='nearest').isel(ensemble_member=1) to select ensamble member
        
        """NEAR SURFACE"""
        #ACCUMULATED PRECIPITATION AMOUNT
        precipitation_amount_acc = ds.precipitation_amount_acc.mean(dim=('x','y'),skipna=True)  
        #AIR PRESSURE AT SEA LEVEL
        air_pressure_at_sea_level = ds.air_pressure_at_sea_level.mean(dim=('x','y'),skipna=True)
        #2M AIR TEMPERATURE
        air_temperature_2m = ds.air_temperature_2m.mean(dim=('x','y'),skipna=True)
     
        
        #2M RH
        #relative_humidity_2m = ds.relative_humidity_2m
        
        #AIR TEMP AT LOWEST LEVEL
        #air_temperature_lowest_level = ds.air_temperature_lowest_level
        
        #10M X-WIND
        #x_wind_10m = ds.x_wind_10m
        
        #10M Y-WIND
        #y_wind_10m = ds.y_wind_10m
        
        # INTEGRAL OF SUFACE DOWNWARD SENSIBLE HEAT FLUX                                  
        #integral_of_surface_dwn_SH = ds.integral_of_surface_downward_sensible_heat_flux_wrt_time
        
        # INTEGRAL OF SURFACE DOWNWELLING LONGWAVE FLUX                                
        #integral_of_surface_downwelling_LW =  ds.integral_of_surface_downwelling_longwave_flux_in_air_wrt_time
       
        # INTEGRAL OF SURFACE DOWNWELLING SHORTWAVE FLUX                                               
        #integral_of_surface_downwelling_SW = ds.integral_of_surface_downwelling_shortwave_flux_in_air_wrt_time 
        
        # INTEGRAL OF SURFACE NET DOWNWARD SHORTWAVE                                                  
        #integral_of_surface_net_downward_SW = ds.integral_of_surface_net_downward_shortwave_flux_wrt_time
       
        # ATMOS BL THICKNESS
        #atmosphere_boundary_layer_thickness = ds.atmosphere_boundary_layer_thickness
     
        """ATM 2D FIELDS"""                                               
        # HIGH TYPE COUD AREA FRACTION
        #high_type_cloud_area_fraction  = ds.high_type_cloud_area_fraction
        
        # LOW TYPE CLOUD ARE FRACTION
        #low_type_cloud_area_fraction  = ds.low_type_cloud_area_fraction
        
        # MEDIUM TYPE CLOUD AREA FRACTION
        #medium_type_cloud_area_fraction = ds.medium_type_cloud_area_fraction 
    
        # CLOUD AREA FRACTION                                                  
        #cloud_area_fraction = ds.cloud_area_fraction 
        
        # FOG AREA FRACTION                                                  
        #fog_area_fraction = ds.fog_area_fraction
                                       
        """ATM 3D FIELDS (925hPa, 850 hPa, 700 hPa, 500 hPa)"""
        
        # AIR TEMPERATURE PRESSURE LEVELS
        #air_temperature_pl = ds.air_temperature_pl
        
        # GEOPOTENTIAL PRESSURE LEVEL                                                   
        #geopotential_pl =  ds.geopotential_pl
        
        # RELATIVE HUMIDITY PRESSURE LEVELS                                                  
        #relative_humidity_pl = ds.relative_humidity_pl
        
        # X WIND PRESSURE LEVELS
        #x_wind_pl = ds.x_wind_p
        
        # Y WIND PRESSURE LEVELS                                                  
        #y_wind_pl = ds.y_wind_pl
        
        #If count is zero, create new Dataset and expand dimension to include 'model_runs'
        if count == 0:
            data = xr.Dataset(data_vars=dict(met_forecast_ref_time = met_forecast_ref_time,\
                                                precipitation_amount_acc=precipitation_amount_acc,\
                                                air_pressure_at_sea_level=air_pressure_at_sea_level,\
                                                air_temperature_2m=air_temperature_2m),\
                                 coords = ds.coords,\
                                 attrs = ds.attrs ) 
            data = data.expand_dims('model_runs')
          
        else:
            #create a dataset with new data
            newdata = xr.Dataset(data_vars=dict(met_forecast_ref_time = met_forecast_ref_time,\
                                                precipitation_amount_acc=precipitation_amount_acc,\
                                                air_pressure_at_sea_level=air_pressure_at_sea_level,\
                                                air_temperature_2m=air_temperature_2m),\
                                 coords = ds.coords,\
                                 attrs = ds.attrs )
            #combine previous data with new data along the 'model_run' dimension at position 'count'
            data = xr.concat([data, newdata], dim='model_runs')
        
        count =+ 1
        
        
        

        
            
                              
    

NameError: name 'ds' is not defined

In [3]:
indx_x = []
indx_y = []
for i in range(len(check_points.x)):
    for j in range(len(check_points.y)):
        dx = abs(check_points.x[i] - da.x)
        dy = abs(check_points.y[j] - da.y)
        distance = np.sqrt(dx**2 + dy**2)
        
        
        if distance <= dist_threshold:
            idx = ds.indexes['x'].get_loc(check_points.x[i],method='nearest')
            indx_x.append(idx)
            idy = ds.indexes['y'].get_loc(check_points.y[j],method='nearest')
            indx_y.append(idy)
        else:
            continue

In [8]:
print(ix,iy)
print('indx_x:',indx_x,'indx_y:',indx_y)



157 352
indx_x: <xarray.DataArray (z: 5)>
array([156, 157, 157, 157, 158])
Dimensions without coordinates: z indx_y: <xarray.DataArray (z: 5)>
array([352, 351, 352, 353, 352])
Dimensions without coordinates: z


In [5]:
indx_x = xr.DataArray(indx_x, dims ='z')
indx_y = xr.DataArray(indx_y, dims='z')
sel_ds = ds.sel(x=indx_x, y=indx_y,method='nearest').mean(dim='z', skipna=True)


<xarray.Dataset>
Dimensions:                                                         (ensemble_member: 10, height0: 1, pressure: 5, time: 67, z: 5, hybrid0: 1, height1: 1, height3: 1, height_above_msl: 1)
Coordinates:
  * ensemble_member                                                 (ensemble_member) int16 ...
  * height0                                                         (height0) float32 ...
  * pressure                                                        (pressure) float32 ...
  * time                                                            (time) datetime64[ns] ...
    x                                                               (z) float32 ...
    y                                                               (z) float32 ...
  * height1                                                         (height1) float32 ...
  * height3                                                         (height3) float32 ...
  * height_above_msl                                                (height_above_msl) float32 ...
  * hybrid0                                                         (hybrid0) float64 ...
    latitude                                                        (z) float64 ...
    longitude                                                       (z) float64 ...
Dimensions without coordinates: z
Data variables: (12/44)
    projection_lambert                                              int32 ...
    ap0                                                             (hybrid0) float64 ...
    b0                                                              (hybrid0) float64 ...
    p00                                                             float64 ...
    forecast_reference_time                                         datetime64[ns] ...
    air_temperature_0m                                              (time, height0, ensemble_member, z) float32 ...
    ...                                                              ...
    snowfall_amount_acc                                             (time, height0, ensemble_member, z) float32 ...
    wind_speed_of_gust                                              (time, height3, ensemble_member, z) float32 ...
    x_wind_10m                                                      (time, height3, ensemble_member, z) float32 ...
    x_wind_pl                                                       (time, pressure, ensemble_member, z) float64 ...
    y_wind_10m                                                      (time, height3, ensemble_member, z) float32 ...
    y_wind_pl                                                       (time, pressure, ensemble_member, z) float64 ...
Attributes: (12/18)
    Conventions:                     CF-1.6
    institution:                     Norwegian Meteorological Institute, MET ...
    creator_url:                     met.no
    summary:                         MEPS (MetCoOp-Ensemble Prediction System...
    source:                          MEPS 2.5km
    min_time:                        2017-01-01 06:00:00Z
    ...                              ...
    license:                         http://met.no/sb5-met.no/Forskning/Infor...
    comment:                         none
    title:                           MEPS subset 2.5km
    NCO:                             "4.6.0"
    history:                         Wed Sep 12 08:53:51 2018 Adding forecast...
    DODS_EXTRA.Unlimited_Dimension:  time

<xarray.DataArray 'relative_humidity_2m' (z: 5)>
array([0.940985, 0.940985, 0.940985, 0.940985, 0.940985], dtype=float32)
Coordinates:
    ensemble_member  int16 0
    time             datetime64[ns] 2017-01-01T06:00:00
    x                (z) float32 57.83 57.83 57.83 57.83 57.83
    y                (z) float32 678.2 678.2 678.2 678.2 678.2
    height1          float32 2.0
    latitude         (z) float64 63.01 63.01 63.01 63.01 63.01
    longitude        (z) float64 15.0 15.0 15.0 15.0 15.0
Dimensions without coordinates: z
Attributes:
    long_name:      Screen level relative humidity (RH2M)
    standard_name:  relative_humidity
    units:          1
    grid_mapping:   projection_lambert
    _ChunkSizes:    [  1   1   1 949 739]


<xarray.Dataset>
Dimensions:                                                         (ensemble_member: 10, height0: 1, pressure: 5, time: 67, hybrid0: 1, height1: 1, height3: 1, height_above_msl: 1)
Coordinates:
  * ensemble_member                                                 (ensemble_member) int16 ...
  * height0                                                         (height0) float32 ...
  * pressure                                                        (pressure) float32 ...
  * time                                                            (time) datetime64[ns] ...
  * height1                                                         (height1) float32 ...
  * height3                                                         (height3) float32 ...
  * height_above_msl                                                (height_above_msl) float32 ...
  * hybrid0                                                         (hybrid0) float64 ...
Data variables: (12/44)
    projection_lambert                                              float64 -...
    ap0                                                             (hybrid0) float64 ...
    b0                                                              (hybrid0) float64 ...
    p00                                                             float64 1...
    forecast_reference_time                                         datetime64[ns] ...
    air_temperature_0m                                              (time, height0, ensemble_member) float32 ...
    ...                                                              ...
    snowfall_amount_acc                                             (time, height0, ensemble_member) float32 ...
    wind_speed_of_gust                                              (time, height3, ensemble_member) float32 ...
    x_wind_10m                                                      (time, height3, ensemble_member) float32 ...
    x_wind_pl                                                       (time, pressure, ensemble_member) float64 ...
    y_wind_10m                                                      (time, height3, ensemble_member) float32 ...
    y_wind_pl                                                       (time, pressure, ensemble_member) float64 ...

In [18]:

sel_ds_1.relative_humidity_2m

<xarray.DataArray 'relative_humidity_2m' (time: 67, height1: 1, ensemble_member: 10)>
array([[[0.9409854 , 0.88942116, 1.0000023 , 0.9339163 , 0.94945   ,
         0.91580427, 0.9569419 , 0.9251591 , 0.99022216, 0.9518471 ]],

       [[0.954459  , 0.98242617, 0.9828824 , 0.97160757, 0.9143094 ,
         0.93861884, 0.9836013 , 0.97127086, 0.948507  , 0.981496  ]],

       [[0.97374594, 0.9430133 , 0.9816303 , 0.9614609 , 0.9751624 ,
         0.9652297 , 0.97652113, 0.9440509 , 0.9592497 , 0.9839982 ]],

       [[0.95984477, 0.90636176, 0.97673273, 0.95627177, 0.968585  ,
         0.95522004, 0.96185684, 0.9433793 , 0.96523046, 0.9860118 ]],

       [[0.94255626, 0.9047901 , 0.9758481 , 0.92973804, 0.9045743 ,
         0.9359937 , 0.83298004, 0.88689137, 0.9501073 , 0.9782154 ]],

       [[0.88289434, 0.89094985, 0.96536434, 0.94777334, 0.9054128 ,
         0.9579972 , 0.8918492 , 0.89078224, 0.8958944 , 0.9542136 ]],

       [[0.9103905 , 0.9089858 , 0.90016365, 0.9498641 , 0.920458  ,
         0.93491066, 0.9493138 , 0.8912698 , 0.89746463, 0.9211768 ]],
...
                nan,        nan,        nan,        nan,        nan]],

       [[0.9353697 , 0.9353844 ,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan]],

       [[0.929602  , 0.93602467,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan]],

       [[0.9254972 , 0.9309126 ,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan]],

       [[0.934622  , 0.93036306,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan]],

       [[0.92690074, 0.93918353,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan]],

       [[0.9484309 , 0.9359485 ,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan]]],
      dtype=float32)
Coordinates:
  * ensemble_member  (ensemble_member) int16 0 1 2 3 4 5 6 7 8 9
  * time             (time) datetime64[ns] 2017-01-01T06:00:00 ... 2017-01-04
  * height1          (height1) float32 2.0